# Test 5 (paper) - Experimento 5 (PT)

Entrenamiento de Mask R-CNN con dataset modificado para emular "Test 5"

### Hiperparametros
* **epoch = 300**
    * steps x epoch = 146 (lotes de imagenes)
    * batch = 2
* optimizador = SGD
* Funcion de perdida = SMOOTHL1LOSS
* Metrica de evaluacion = mAP (IoU >= 0.5)
* Mini-mask shape: 56x56
* RPN anchor scales: (32, 64, 128, 256, 512)
* Tasa de aprendizaje: 0.001
* imagenes = 292
    * entrenamiento 81% = 237
    * validacion 19% = 55
* etiquetas = 9140
* **resolucion = 1920 x 1080**
* etiquetas = bounding box formato VOC XML
* **numero de clases = 2 (arandano, arandano-maduro)**
* **data augmentation = false**


## Comprobar directorio principal

In [1]:
!pwd && ls -l

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport/model-training
total 1772
drwxr-xr-x 4 root root    4096 Aug  7 06:10 build
drwxr-xr-x 2 root root    4096 Aug  7 06:10 dist
drwxr-xr-x 2 root root    4096 Aug  8 03:15 mask_rcnn.egg-info
-rw-r--r-- 1 root root  162962 Sep 13 07:31 master-test_5.ipynb
-rw-r--r-- 1 root root   31908 Sep 13 07:49 master-test_5_fase_2.ipynb
-rw-r--r-- 1 root root  110113 Sep 13 07:33 master-test_6.ipynb
-rw-r--r-- 1 root root   17222 Sep 13 07:36 master-test_6_fase_2.ipynb
-rw-r--r-- 1 root root 1395023 Sep 13 01:41 master-v2.ipynb
-rw-r--r-- 1 root root   71399 Aug 17 21:33 master.ipynb
drwxr-xr-x 4 root root    4096 Sep 13 07:49 mrcnn


# Importar bibliotecas

In [2]:
# bibliotecas basicas
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# bibliotecas avanzadas 
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# bibliotecas mask rcnn 
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# biblioteca matplotlib 
import matplotlib.pyplot as plt

# bibliotecas numpy 
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# bibliotecas keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# ignorar alertas de elementos que seran descontinuados
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
#plt.show()

import imgaug.augmenters as iaa

# Fase 2 - Entrenamiento con dos clases y etiquetas de Bounding Box

Entrenamiento del modelo con 2 clases:
arandano, arandano-maduro
Las imágenes estan etiquetadas con bounding box de la misma manera que en Fase 1.

In [3]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [4]:
class DatasetArandanos(Dataset):
    
    # load_dataset function is used to load the train and test dataset
    def load_dataset(self, dataset_dir, is_train=True):
        
        # we use add_class for each class in our dataset and assign numbers to them. 0 is background
        # self.add_class('source', 'class id', 'class name')
        self.add_class("dataset", 1, "arandano")
        self.add_class("dataset", 2, "arandano-maduro")
        #self.add_class("dataset", 3, "Level-3")
        #self.add_class("dataset", 4, "Level-4")
        
        # we concatenate the dataset_dir with /images and /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train will be true if we our training our model and false when we are testing the model
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4] # used to skip last 4 chars which is '.jpg' (class_id.jpg)
            
            # if is_train is True skip all images with id greater than and equal to 160
             # roughly 80% of dataset for training
            if is_train and int(image_id) >= 41590 :
                #print("image_id: ", image_id)
                continue
             
            # if is_train is not True skip all images with id less than 420
            if not is_train and int(image_id) < 41590:
                continue
            
            # img_path and ann_path variables are defined
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # using add_image function we pass image_id, image_path and ann_path so that the current
            # image is added to the dataset for training
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # function used to extract bouding boxes from annotated files
    def extract_boxes(self, filename):

        # you can see how the images are annotated we extracrt the width, height and bndbox values

        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>

        # </annotation>
        
        # used to parse the .xml files
        tree = ElementTree.parse(filename)
        
        # to get the root of the xml file
        root = tree.getroot()
        
        # we will append all x, y coordinated in boxes
        # for all instances of an onject
        boxes = list()
        
        # we find all attributes with name bndbox
        # bndbox will exist for each ground truth in an image
        for box in root.findall('.//object'):
            name = box.find('name').text
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
            
            # I have included this line to skip any un-annotated images
            if name=='arandano' or name=='arandano-maduro':
                boxes.append(coors)

        # extract width and height of the image
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # return boxes-> list, width-> int and height-> int 
        return boxes, width, height
    
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances        
    def load_mask(self, image_id):
        
        # info points to the current image_id
        info = self.image_info[image_id]
        
        # we get the annotation path of image_id which is dataset_dir/annots/image_id.xml
        path = info['annotation']
        
        # we call the extract_boxes method(above) to get bndbox from .xml file
        boxes, w, h = self.extract_boxes(path)
        
        # we create len(boxes) number of masks of height 'h' and width 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = list()
        
        # we loop over all boxes and generate masks (bndbox mask) and class id for each instance
        # masks will have rectange shape as we have used bndboxes for annotations
        # for example: if 2.jpg have four objects we will have following masks and class_ids
        # 000000000 000000000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000000000 022200000 000000000 000000000
        #    1         2          3         1<- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            
            # box[4] will have the name of the class for a particular damage
            if (box[4] == 'arandano'):
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('arandano'))
            elif(box[4] == 'arandano-maduro'):
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('arandano-maduro')) 
                
        # return masks and class_ids as array
        return masks, asarray(class_ids, dtype='int32')
    
    # this functions takes the image_id and returns the path of the image
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']




Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DEVICE                         /gpu:0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.8
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE 

In [ ]:
# damage configuration class, you can change values of hyper parameters here
class ConfigArandanos(Config):

    # nombre de la configuracion
    NAME = "arandano_cfg_test_5_fase_2_2_"    
    
# clase arandano + clase background 
    NUM_CLASSES = 1 + 2
    
    # pasos por epoch y confianza minima    # STEPS_PER_EPOCH = cantidad de lotes/batchs
    #STEPS_PER_EPOCH = 73   # por epoch se entrenaran 73 lotes de 4 imagenes, dataset = 292
    STEPS_PER_EPOCH = 146
    
    # tasa de aprendizaje y momentum
    LEARNING_RATE=0.001
    LEARNING_MOMENTUM = 0.8
    
    # penalización de regularización
    WEIGHT_DECAY = 0.0001
    
    # el tamaño de la imagen está controlado por este parámetro
    IMAGE_MIN_DIM = 512
    
    # pasos de validación
    VALIDATION_STEPS = 50
    
    # número de regiones de interés generadas por imagen
    Train_ROIs_Per_Image = 200
    
    # escala de anclas RPN y proporciones (ratios) para encontrar la ROI
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)    # Longitud del lado del ancla cuadrada, en píxeles 
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]   # Proporciones de anclas por cada celda (ancho/alto). Un valor de 1 representa un ancla cuadrada y 0,5 es un ancla ancha 
    
    MINI_MASK_SHAPE = (56, 56)
    
    
config = ConfigArandanos()
config.display()

### Entrenamiento

In [5]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [6]:
#pesos = 'mask_rcnn_coco.h5'
#pesos = 'mask_rcnn_damage_cfg_0049.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'

conjunto_datos = 'customImages/test_5_fase_2'


In [7]:
# cargar dataset de entrenamiento
train_set = DatasetArandanos()
train_set.load_dataset(conjunto_datos, is_train=True)
train_set.prepare()

# cargar dataset de test 
test_set = DatasetArandanos()
test_set.load_dataset(conjunto_datos, is_train=False)
test_set.prepare()

# preparar la configuración llamando a la clase de configuración definida por el usuario
config = ConfigArandanos()

# definir el modelo
with tf.device(config.DEVICE):
    model = MaskRCNN(mode='training', model_dir='./', config=config)

# cargar pesos del modelo 
weights_path = pesos

# cargar los pesos del modelo
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

#augmentation = iaa.Sequential(iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sometimes(5/6,iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sequential([iaa.Fliplr(0.5), iaa.Flipud(0.5)])
#augmentation = iaa.Sometimes(5/6, iaa.Fliplr(0.5), iaa.Flipud(0.5))
augmentation = iaa.Sometimes(0.5, iaa.Fliplr(0.5), iaa.Flipud(0.5))

# start the training of model
# you can change epochs and layers (head or all)
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=300, layers='all', augmentation=augmentation)



Starting at epoch 0. LR=0.001

Checkpoint Path: ./arandano_cfg_test_5_fase_2_20220913T0749/mask_rcnn_arandano_cfg_test_5_fase_2__{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (C

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/usr/local/lib/python3.8/dist-packages/imgaug/augmenters/base.py:49: SuspiciousSingleImageShapeWarning: You provided a numpy array of shape (1024, 1024, 98) as a single-image augmentation input, which was interpreted as (H, W, C). The last dimension however has a size of >=32, which indicates that you provided a multi-image array with shape (N, H, W) instead. If that is the case, you should use e.g. augmenter(imageS=<your input>) or augment_imageS(<your input>). Otherwise your multi-image input will be interpreted as a single image during augmentation.
  ia.warn(


Epoch 1/300


/usr/local/lib/python3.8/dist-packages/imgaug/augmenters/base.py:49: SuspiciousSingleImageShapeWarning: You provided a numpy array of shape (1024, 1024, 52) as a single-image augmentation input, which was interpreted as (H, W, C). The last dimension however has a size of >=32, which indicates that you provided a multi-image array with shape (N, H, W) instead. If that is the case, you should use e.g. augmenter(imageS=<your input>) or augment_imageS(<your input>). Otherwise your multi-image input will be interpreted as a single image during augmentation.
  ia.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("train

/usr/local/lib/python3.8/dist-packages/imgaug/augmenters/base.py:49: SuspiciousSingleImageShapeWarning: You provided a numpy array of shape (1024, 1024, 34) as a single-image augmentation input, which was interpreted as (H, W, C). The last dimension however has a size of >=32, which indicates that you provided a multi-image array with shape (N, H, W) instead. If that is the case, you should use e.g. augmenter(imageS=<your input>) or augment_imageS(<your input>). Otherwise your multi-image input will be interpreted as a single image during augmentation.
  ia.warn(
/usr/local/lib/python3.8/dist-packages/imgaug/augmenters/base.py:49: SuspiciousSingleImageShapeWarning: You provided a numpy array of shape (1024, 1024, 38) as a single-image augmentation input, which was interpreted as (H, W, C). The last dimension however has a size of >=32, which indicates that you provided a multi-image array with shape (N, H, W) instead. If that is the case, you should use e.g. augmenter(imageS=<your inpu

...

Epoch 201/300
146/146 [==============================] - 445s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1298 - rpn_class_loss: 6.5024e-04 - rpn_bbox_loss: 0.0082 - mrcnn_class_loss: 0.0285 - mrcnn_bbox_loss: 0.0123 - mrcnn_mask_loss: 0.0802 - val_loss: 8.4703 - val_rpn_class_loss: 4.5579 - val_rpn_bbox_loss: 0.9384 - val_mrcnn_class_loss: 1.5988 - val_mrcnn_bbox_loss: 0.3685 - val_mrcnn_mask_loss: 1.0068
Epoch 202/300
146/146 [==============================] - 444s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1341 - rpn_class_loss: 8.5633e-04 - rpn_bbox_loss: 0.0091 - mrcnn_class_loss: 0.0297 - mrcnn_bbox_loss: 0.0126 - mrcnn_mask_loss: 0.0818 - val_loss: 7.5208 - val_rpn_class_loss: 3.9579 - val_rpn_bbox_loss: 0.8391 - val_mrcnn_class_loss: 1.4676 - val_mrcnn_bbox_loss: 0.3382 - val_mrcnn_mask_loss: 0.9181
Epoch 203/300
146/146 [==============================] - 470s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1339 - rpn_class_loss: 9.8102e-04 - rpn_bbox_loss: 0.009

Epoch 221/300
146/146 [==============================] - 434s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1265 - rpn_class_loss: 7.3117e-04 - rpn_bbox_loss: 0.0088 - mrcnn_class_loss: 0.0280 - mrcnn_bbox_loss: 0.0117 - mrcnn_mask_loss: 0.0773 - val_loss: 8.4538 - val_rpn_class_loss: 4.4794 - val_rpn_bbox_loss: 0.8779 - val_mrcnn_class_loss: 1.5011 - val_mrcnn_bbox_loss: 0.3821 - val_mrcnn_mask_loss: 1.2132
Epoch 222/300
146/146 [==============================] - 479s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1287 - rpn_class_loss: 6.8667e-04 - rpn_bbox_loss: 0.0093 - mrcnn_class_loss: 0.0289 - mrcnn_bbox_loss: 0.0117 - mrcnn_mask_loss: 0.0780 - val_loss: 8.1913 - val_rpn_class_loss: 4.5084 - val_rpn_bbox_loss: 0.9802 - val_mrcnn_class_loss: 1.4249 - val_mrcnn_bbox_loss: 0.3529 - val_mrcnn_mask_loss: 0.9250
Epoch 223/300
146/146 [==============================] - 424s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1284 - rpn_class_loss: 7.7637e-04 - rpn_bbox_loss: 0.009

Epoch 241/300
146/146 [==============================] - 504s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1181 - rpn_class_loss: 6.7096e-04 - rpn_bbox_loss: 0.0078 - mrcnn_class_loss: 0.0248 - mrcnn_bbox_loss: 0.0109 - mrcnn_mask_loss: 0.0740 - val_loss: 8.1888 - val_rpn_class_loss: 4.1972 - val_rpn_bbox_loss: 0.7562 - val_mrcnn_class_loss: 1.6837 - val_mrcnn_bbox_loss: 0.3601 - val_mrcnn_mask_loss: 1.1916
Epoch 242/300
146/146 [==============================] - 447s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1167 - rpn_class_loss: 6.0772e-04 - rpn_bbox_loss: 0.0077 - mrcnn_class_loss: 0.0256 - mrcnn_bbox_loss: 0.0106 - mrcnn_mask_loss: 0.0721 - val_loss: 8.4245 - val_rpn_class_loss: 4.1510 - val_rpn_bbox_loss: 0.7366 - val_mrcnn_class_loss: 1.8621 - val_mrcnn_bbox_loss: 0.3901 - val_mrcnn_mask_loss: 1.2846
Epoch 243/300
146/146 [==============================] - 457s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1157 - rpn_class_loss: 7.7255e-04 - rpn_bbox_loss: 0.007

Epoch 261/300
146/146 [==============================] - 488s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1183 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0084 - mrcnn_class_loss: 0.0260 - mrcnn_bbox_loss: 0.0105 - mrcnn_mask_loss: 0.0723 - val_loss: 8.4821 - val_rpn_class_loss: 4.5173 - val_rpn_bbox_loss: 0.8753 - val_mrcnn_class_loss: 1.5931 - val_mrcnn_bbox_loss: 0.3650 - val_mrcnn_mask_loss: 1.1314
Epoch 262/300
146/146 [==============================] - 485s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1097 - rpn_class_loss: 4.4192e-04 - rpn_bbox_loss: 0.0068 - mrcnn_class_loss: 0.0242 - mrcnn_bbox_loss: 0.0097 - mrcnn_mask_loss: 0.0685 - val_loss: 8.5944 - val_rpn_class_loss: 4.4904 - val_rpn_bbox_loss: 0.8526 - val_mrcnn_class_loss: 1.6712 - val_mrcnn_bbox_loss: 0.3756 - val_mrcnn_mask_loss: 1.2046
Epoch 263/300
146/146 [==============================] - 435s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1094 - rpn_class_loss: 8.2814e-04 - rpn_bbox_loss: 0.0068 - 

Epoch 281/300
146/146 [==============================] - 449s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1012 - rpn_class_loss: 5.3861e-04 - rpn_bbox_loss: 0.0062 - mrcnn_class_loss: 0.0221 - mrcnn_bbox_loss: 0.0090 - mrcnn_mask_loss: 0.0634 - val_loss: 7.7098 - val_rpn_class_loss: 4.0017 - val_rpn_bbox_loss: 0.7380 - val_mrcnn_class_loss: 1.4938 - val_mrcnn_bbox_loss: 0.3727 - val_mrcnn_mask_loss: 1.1037
Epoch 282/300
146/146 [==============================] - 465s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1086 - rpn_class_loss: 6.0132e-04 - rpn_bbox_loss: 0.0074 - mrcnn_class_loss: 0.0237 - mrcnn_bbox_loss: 0.0097 - mrcnn_mask_loss: 0.0672 - val_loss: 8.6683 - val_rpn_class_loss: 4.5548 - val_rpn_bbox_loss: 0.7896 - val_mrcnn_class_loss: 1.7328 - val_mrcnn_bbox_loss: 0.3705 - val_mrcnn_mask_loss: 1.2206
Epoch 283/300
146/146 [==============================] - 450s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1006 - rpn_class_loss: 4.2584e-04 - rpn_bbox_loss: 0.005